In [263]:
import pandas as pd
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.use('pgf')
'pgf'
import numpy as np
from sklearn.metrics import mean_squared_error
from pathlib import Path
import os
import pickle
from models.gan.gan import generator, discriminator, StockTimeGan
from utils.log import prepare_logger
from pathlib import Path
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime

plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
    "figure.figsize": (30, 10), 
    "figure.dpi": 600
})


logger = prepare_logger("INFO")
# %load_ext blackcellmagic

# %%black for formatting


In [264]:
TICKER = "TTWO"
MODEL_VERSION = 0.1

In [265]:
load_path = Path(os.path.abspath("")).parents[0] / "data" / "scaled_data"
models_path = Path(os.path.abspath("")).parents[0] / "models" / "gan" / "versions"

with open(load_path / f"data_{TICKER}.pickle", "rb") as handle:
    data = pickle.load(handle)

with open(load_path / f"scalers_{TICKER}.pickle", "rb") as handle:
    scalers = pickle.load(handle)

with open(models_path / f"model_{MODEL_VERSION}_{TICKER}.pickle", "rb") as handle:
    model_data = pickle.load(handle)


In [266]:
test_preds = scalers["y_scaler"].inverse_transform(model_data["test_preds"])
test_true = scalers["y_scaler"].inverse_transform(model_data["actual_values"])


In [267]:
# plt.plot(data["Y_preds_real_list_train"])
# plt.show()

In [268]:
# plt.plot(data["Y_preds_real_list_test"])
# plt.show()

In [269]:
# full_y = list(scalers["y_scaler"].inverse_transform(data["Y_preds_real_list_train"]))
# full_y.extend((list(scalers["y_scaler"].inverse_transform(data["Y_preds_real_list_test"]))))

# plt.plot(full_y)
# plt.axvline(535, label='Train/Test boundary')
# plt.title('EA Close price')
# plt.legend()
# plt.show()

In [270]:
data["Y_preds_real_list_train"].shape

(535, 1)

In [271]:
data["X_list_train"].shape

(535, 30, 19)

In [272]:
data["Y_preds_real_list_test"].shape

(178, 1)

In [273]:
mean_absolute_error(y_true=test_true, y_pred=test_preds)


1.3584927976801153

In [274]:
np.sqrt(mean_squared_error(y_true=test_true, y_pred=test_preds))


1.8866734342294273

In [275]:
df = pd.DataFrame(
    {
        "predictions": test_preds.reshape(
            test_preds.shape[0],
        ),
        "actual_values": test_true.reshape(
            test_true.shape[0],
        ),
    }
)


In [276]:
stock_path = Path(os.path.abspath("")).parents[0] / "data" / "stock_data" / f'{TICKER}.csv'
stock_df = pd.read_csv(stock_path)

In [277]:
stock_df["Date"] = pd.to_datetime(stock_df["Date"])
dates = (
    stock_df[
        stock_df["Date"]
        >= datetime.datetime(year=2019, month=1, day=1) + datetime.timedelta(days=30)
    ]
    .tail(178)["Date"]
    .values
)


In [278]:
fig_save_path = Path(os.path.abspath("")).parents[0] / "paper"

In [279]:
# fig_save_path = Path(os.path.abspath("")).parents[0] / "paper"
# plt.plot(
#     dates,
#     test_preds.reshape(
#         test_preds.shape[0],
#     ),
#     color="blue",
#     label="Predicted price",
# )
# plt.plot(
#     dates,
#     test_true.reshape(
#         test_true.shape[0],
#     ),
#     color="red",
#     label="Actual price",
# )
# plt.title(f"{TICKER} actual price vs predicted price", size=25)
# plt.xlabel("Date", size=20)
# plt.ylabel("Valuation", size=20)
# plt.xticks(fontsize=15)
# plt.yticks(fontsize=15)
# plt.legend(prop={'size': 15})
# plt.show()
# plt.savefig(fig_save_path / f'{TICKER}_prices.pgf')


In [280]:
test_true.mean()


83.6483415539345

In [281]:
np.median(test_true)


83.47000122070312

In [282]:
np.std(test_true)


11.434061630252828

In [283]:
results = {"1000_epochs_no_sentiment": {"mea": 1.2186, "rmse": 1.7288}}


In [284]:
plt.plot(
    model_data['train_history']['gen_loss'],
    color="blue",
    label="Generator loss",
)
plt.plot(
    model_data['train_history']['disc_loss'],
    color="red",
    label="Discriminator Loss",
)
plt.title(f"{TICKER} generator and discriminator loss", size=25)
plt.xlabel("Epoch", size=20)
plt.ylabel("Loss", size=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(prop={'size': 15})
plt.legend()
plt.show()
plt.savefig(fig_save_path / f'{TICKER}_loss.pgf')

C:\Users\kuba1\AppData\Local\Temp\ipykernel_16716\2705492668.py:18: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [285]:
model_data['train_history'].keys()

dict_keys(['gen_loss', 'disc_loss', 'real_y', 'pred_y'])

O czym napisac: 
- Spotkanie 
- Problemy ze stabilizacja modelu 
- Wywalenie dense, dodawanie i wywalanie sentymentu 
- Co z innymi modelami 
- Co jeszcze potrzeba 
- Wykresy? 
- Czy inne firmy 